In [1]:
%reload_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False

In [15]:
import pandas as pd
import numpy as np

# Load data from google sheet
sheet_id = "1QaXBrhDY8tyF9cIBjejDTGMiv5vDUQG0KQ7iVapGQoY"
gid = 1987847080
link = f"https://docs.google.com/feeds/download/spreadsheets/Export?key={sheet_id}&exportFormat=csv&gid={gid}"
envs = pd.read_csv(link)

# Preprocessing

# Add house type icons
def emojify(row):  
    house_type = row["House type"]
    if house_type == "residential":
        return "|home|"
    elif house_type == "office":
        return "|office|"
    elif house_type == "industrial":
        return "|industry|"
    return "other"

def new_type(row):  
    house_type = row["House type"]
    if house_type == "residential":
        return "residential building"
    elif house_type == "office":
        return "office building"
    elif house_type == "industrial":
        return "data center"
    return "data center"

envs['Type'] = envs.apply(lambda row: emojify(row), axis=1)
envs['new_type'] = envs.apply(lambda row: new_type(row), axis=1)

# Make sure each env has gym
envs['Gym'] = envs['Gym'].replace('', np.NaN)
envs['Gym'] = envs['Gym'].fillna(method='ffill',axis=0)






In [16]:
# Create main environment list

def create_main_list(filepath="../../docs/envs/envs_list.rst"):

    rst_out = ""

    for i, gym in enumerate(envs['Gym'].unique()):
        gym_envs = envs[envs['Gym']==gym]

        rst_out += f":{gym}:\n"

        for index, row in gym_envs.iterrows():
            env = row["Environment"]
            symbol = row["Type"]
            #rst_out += f"    - `{env}`_ {symbol}\n"
            rst_out += f"    - `{env}`_\n"

        # Add hline between gyms
        if i < len(envs['Gym'].unique()) - 1:
            rst_out += "\n----\n\n"

    # Add image links
    rst_out += """
    .. |office| image:: https://raw.githubusercontent.com/tabler/tabler-icons/master/icons/building-skyscraper.svg
    .. |home| image:: https://raw.githubusercontent.com/tabler/tabler-icons/master/icons/home.svg
    .. |industry| image:: https://raw.githubusercontent.com/tabler/tabler-icons/master/icons/building-factory.svg
    """

    with open(filepath, 'w') as file:
        file.write(rst_out)


create_main_list(filepath="../../docs/envs/envs_list.rst")

In [17]:
def create_env_descriptions():
    for _, gym in enumerate(envs['Gym'].unique()):
        env_descr_file = f"../../docs/envs/{gym}_descriptions.rst"
        rst_out = ""
        gym_envs = envs[envs['Gym']==gym]

        for _, row in gym_envs.iterrows():
            env = row["Environment"]
            rst_out += f"\n\n.. _env-{env}: \n\n"
            rst_out += f"``{env}``\n"
            rst_out += '"' * (len(env) + 4) + "\n\n"
            #rst_out += f":Type: {row['new_type']} ({row['Type']})\n"
            rst_out += f":Type: {row['new_type']}\n"
            rst_out += f":More info: `framework docs <{row['Original docs']}>`_\n"

        with open(env_descr_file, 'w') as file:
            file.write(rst_out)

create_env_descriptions()
    